Load all libraries

In [3]:
import pandas as pd
import nltk

Read Dataset

In [5]:
res_single_df = pd.read_csv('../restaurant-single-categories.csv')
res_multiple_df = pd.read_csv('../restaurant-multiple-categories.csv')

lap_single_df = pd.read_csv('../laptop-single-categories.csv')
lap_multiple_df = pd.read_csv('../laptop-multiple-categories.csv')

Dataset Exploration

In [6]:
res_single_df.head()

,Unnamed: 0,reviewID,sentenceID,review,category,polarity
0,0,1004293,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,negative
1,1,1004293,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,negative
2,2,1004293,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,negative
3,3,1004293,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,negative
4,4,1004293,1004293:4,"After all that, they complained to me about th...",SERVICE#GENERAL,negative
